## Biot's equation

This notebooks aims at testing the effect of different boundary conditions for Biot's equation on a 2D doughnut mesh. 
Since large pressure gradients withn the brain are very unlikely, we are especially interested in physically meaningiful combinations of boundary conditions, that lead to smooth pressure results. Additionally, it is interesting to see why certain BCs result in large pressure gradients and improve the physical understanding of the underlying mechanism.

The quasi-static equations for Biot's model are:

$$
- \nabla \cdot 2 \mu \epsilon (u) - \nabla \lambda \nabla u I + \alpha \nabla p = f \quad \text{in $\Omega \times (0,T]$ (momentum equation)}
$$
$$
c \frac{\partial p}{\partial t} + \alpha \frac{\nabla \cdot u}{\partial t} - \nabla \cdot (K \nabla p) = g \quad \text{ in $\Omega \times (0,T]$ (continuity equation)}
$$

Suitable boundary conditions might be either No-flux (Neumann) or Dirichlet BCs for the pressure and a total stress BC for the displacement:

$$
(2 \mu \epsilon (u) + \lambda \nabla \cdot u I - \alpha p I) \cdot n = - p_{obs} n \quad \text{on $\Gamma \times (0,T]$}
$$

$$
\nabla p \cdot n = 0 \quad \text{on $\Gamma_N \times (0,T]$} \\
p = p_{obs} \quad \text{on $\Gamma_D \times (0,T]$}
$$

As inital conditions, we assume the system to be at rest:

$$
u(x, 0) = 0 \text{ in $\Omega$}\\
p(x, 0) = 0 \text{ in $\Omega$}
$$


In [ ]:
%load_ext autoreload
%autoreload 2
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
#from fenics_adjoint import *
from braininversion.BiotSolver import solve_biot
from braininversion.meshes import generate_doughnut_mesh
from braininversion.PlottingHelper import (plot_pressures_and_forces_timeslice, 
                            plot_pressures_and_forces_cross_section,
                            extract_cross_section, style_dict)
set_log_level(13)


T = 1.0           # final time
num_steps = 20    # number of time steps
nx = 100
dt = T / num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability 15*(1e-9)**2
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-8         # storage coefficent
alpha = 1.0         # Biot-Willis coefficient

# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha

brain_radius = 0.1
ventricle_radius = 0.03
N = 10

mesh, bm = generate_doughnut_mesh(brain_radius, ventricle_radius, N)

mmHg2Pa = 132.32
freq = 1.0 
A = 2*mmHg2Pa
p_obs = Expression("A*sin(2*pi*f*t)", A=A,f=freq,t=0,degree=2)
p_obs_lin = Expression("A*sin(2*pi*f*t)*(0.8 + 2*sqrt(x[0]*x[0] + x[1]*x[1]))", A=A,f=freq,t=0,degree=2)
#p_obs_lin = p_obs
n = FacetNormal(mesh)

g = Constant(0.0)
f = Constant([0.0, 0.0])

In [ ]:
# total stress at both boundaries, no flux BC for pressure

boundary_conditions_u = {2:{"Neumann":n*p_obs_lin},
                         1:{"Neumann":n*Constant(0.0)},
                         #2:{"Neumann":n*p_obs_lin},
                        } 
                         
boundary_conditions_p = { 2:{"Neumann":Constant(0.0)},
                          1:{"Neumann":Constant(0.0)},
                         #1:{"Dirichlet":p_obs_lin},
                         }
u_nullspace = True

solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0,
                      solver_type="krylov")
solution = [s.copy() for s in solution]

In [ ]:
V = FunctionSpace(mesh, "CG", 1)
displ = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]
pressure = [s.split()[2] for s in solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]

x_coords = np.linspace(ventricle_radius, brain_radius, 100)
slice_points = [Point(x, 0.0) for x in x_coords]

pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,
             "prescribed outer pressure"   : extract_cross_section(p_obs_lin, slice_points, times)/mmHg2Pa,}

displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}
style_dict["negative_total_pressure"] = {"ls":":", "lw":3, "color":"firebrick"}
style_dict["fluid_pressure"] = {"ls":":", "lw":3, "color":"orange"}
style_dict["displacement [m]"] = {"ls":"-.", "lw":3, "color":"green"}
style_dict["p_opt"] = {"ls":"-.", "lw":3, "color":"blue"}
style_dict["prescribed outer pressure"] = {"ls":"-.", "lw":1, "color":"navy"}


In [ ]:
#plt.figure(figsize=(9,7))
#plot(mesh)
#for p in slice_points:
#    plt.scatter(p.array()[0],p.array()[1], s=20, color="red")

In [ ]:
for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [20, 40 ,60, 80]:
    plot_pressures_and_forces_timeslice(pressures, displacement, i, times)
    plt.suptitle(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")

In [ ]:
assemble(div(displ[0])*dx)

In [ ]:
for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plt.figure(figsize=(9,7))
    c = plot(pressure[i]/mmHg2Pa)
    plt.colorbar(c)
    plt.title(f"t = {times[i]:.3f} s")

In [ ]:
# total stress at both boundaries, no flux BC for pressure

boundary_conditions_u = {2:{"Neumann":n*p_obs},
                         1:{"Neumann":n*Constant(0.0)},
                        } 
                         
boundary_conditions_p = { 1:{"Neumann":Constant(0.0)},
                          2:{"Neumann":Constant(0.0)},
                         }
solution = solve_biot(mesh, f, g, T, num_steps, material_parameter,
                      bm, boundary_conditions_p,
                      bm, boundary_conditions_u,
                      u_nullspace=u_nullspace, theta=1.0)
solution = [s.copy() for s in solution]

V = FunctionSpace(mesh, "CG", 1)
displ = [s.split()[0] for s in solution]
total_pressure = [s.split()[1] for s in solution]
pressure = [s.split()[2] for s in solution]
lmbda_div_u = [project(material_parameter["lmbda"]*div(u), V) for u in displ]


pressures = {"negative_total_pressure" : extract_cross_section(total_pressure, slice_points)/mmHg2Pa,
             "fluid_pressure" : extract_cross_section(pressure, slice_points)/mmHg2Pa,
             "lambda_div_u"   : extract_cross_section(lmbda_div_u, slice_points)/mmHg2Pa,}
displacement = {"displacement [m]": extract_cross_section(displ, slice_points),}


In [ ]:
for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plot_pressures_and_forces_cross_section(pressures, displacement, i, x_coords)
    plt.suptitle(f"t = {times[i]:.3f} s")

In [ ]:
for i in [0, 4, 8, 12, 16]: #range(num_steps): 
    plt.figure(figsize=(9,7))
    c = plot(pressure[i])
    plt.colorbar(c)
    plt.title(f"t = {times[i]:.3f} s")

In [ ]:
list_lu_solver_methods()